## 05_AtlasComprobacionDescargas
Autor: Carlos Wong <br/>
email: carlosawongca@gmail.com <br/>
Fecha: 2023_04_19 <br/>
Objetivo: Comprobación trazados faltantes <br/>
Entrada: 'content.xlsx' <br/>
Salida: - <br/>
Propósito: Prueba edición automática de emails <br/>

In [1]:
import pandas as pd
import openpyxl

In [1]:
# Ruta del archivo Excel
ruta_archivo = r'\trazados_STa.xlsx'

# Leer el archivo Excel y crear un diccionario vacío para almacenar los dataframes de cada hoja
dataframes_por_hoja = {}
excel_file = pd.ExcelFile(ruta_archivo)
for nombre_hoja in excel_file.sheet_names:
    # Leer la hoja y seleccionar las columnas a partir de la tercera columna
    hoja = pd.read_excel(ruta_archivo, sheet_name=nombre_hoja, header=1)
    if 'Codificación Común' not in hoja.columns:
        continue  # Saltar esta hoja si no existe la columna 'Codificación Común'
    hoja = hoja.iloc[:, 2:]
    
    # Eliminar filas con valores nulos
    hoja = hoja.dropna(subset=['Codificación Común'])
    
    # Agregar el dataframe a nuestro diccionario con el nombre de la hoja como clave
    dataframes_por_hoja[nombre_hoja] = hoja

# Crear el dataframe df_descargado con las claves de las hojas descargadas
trazados_descargados = pd.DataFrame(list(dataframes_por_hoja.keys()), columns=['Claves'])

# Leer el archivo Excel 'Seguimiento_CCVV_HUAWEI_S.TA_RI.xlsx'
df = pd.read_excel(r'\Seguimiento_CCVV_HUAWEI_S.TA_RI.xlsx', header=1, sheet_name='RI_FO')

# Filtrar las filas de la columna 'ADMINISTRATIVO' que cumplen cierta condición
admin_list_filtered_str = [str(i) for i in df.loc[df['ADMINISTRATIVO'].isin(df['ADMINISTRATIVO'].dropna().tolist()) & df['TipoDeCorte'].str.contains('Corte en caliente', na=False), 'ADMINISTRATIVO'].tolist()]

# Crear el dataframe df_total con las claves filtradas
df_total = pd.DataFrame(admin_list_filtered_str, columns=['Claves'])

# Obtener las claves faltantes
faltantes = set(admin_list_filtered_str) - set(trazados_descargados['Claves'].tolist())

# Convertir las claves faltantes a una lista y crear el dataframe df_faltantes con ellas
df_faltantes = pd.DataFrame({'Faltantes': list(faltantes)})

# Guardar el dataframe df_faltantes como un archivo de Excel
df_faltantes.to_excel(r'\trazados_STa_faltantes.xlsx', index=False)
